## Run Ollama in Colab
<a target="_blank" href="https://colab.research.google.com/github/LiorGazit/agentic_actions_locally_hosted/blob/main/run_ollama_in_colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> (pick a GPU Colab session for fastest computing)  

[List of available Ollama LLMs.](https://ollama.com/library)  
Note: This code will run in Colab but not in Windows. The reason is the Ollama setup. I do believe it would run on Linux in general, but haven't experimented outside of Google Colab.  

```
Disclaimer: The content and ideas presented in this notebook are solely those of the author, Lior Gazit, and do not represent the views or intellectual property of the author's employer.
```

Importing necessary functions:  

If this notebook is run outside of the repo's codes, get the necessary code from the remote repo:

In [1]:
import os
import requests

# If the module isn't already present (e.g. in Colab), fetch it from GitHub
if not os.path.exists("spin_up_LLM.py") or not os.path.exists("monitor_resources.py"):
    url = "https://raw.githubusercontent.com/LiorGazit/agentic_actions_locally_hosted/refs/heads/main/spin_up_LLM.py"
    resp = requests.get(url)
    resp.raise_for_status()
    with open("spin_up_LLM.py", "w") as f:
        f.write(resp.text)
    print("Downloaded spin_up_LLM.py from GitHub")
    url = "https://raw.githubusercontent.com/LiorGazit/agentic_actions_locally_hosted/refs/heads/main/monitor_resources.py"
    resp = requests.get(url)
    resp.raise_for_status()
    with open("monitor_resources.py", "w") as f:
        f.write(resp.text)
    print("Downloaded monitor_resources.py from GitHub")

Downloaded spin_up_LLM.py from GitHub
Downloaded monitor_resources.py from GitHub


In [2]:
from spin_up_LLM import spin_up_LLM
from monitor_resources import start_resource_monitoring

## Run either local LLM for free or remote LLM by OpenAI using your private API key
Set the `mode` variable for your choice.  

In [3]:
mode = "remote"   # If set to "remote", you must paste API key below!

if mode == "local":
  llm_name = "gemma3"
  llm_provider = None
elif mode == "remote":
  # Choose your model name and mode
  llm_name = "gpt-4o"
  llm_provider = "openai"
  os.environ["OPENAI_API_KEY"] = "INSERT OPENAI API KEY HERE"

# Toggle monitoring on/off:
monitor_resources = True

In [4]:
# Sping up an LLM:
model = spin_up_LLM(chosen_llm=llm_name, local_or_remote=mode, provider=llm_provider)

🚀 Setting up remote OpenAI model…
Installing langchain-openai...
All done setting up OpenAI LLM.



In [5]:
# Resource monitoring (via monitor_resources.py)
if monitor_resources:
    # logs for 1h, every 10s, into 'resource_usage.log'
    monitor_thread = start_resource_monitoring(
        duration=3600,
        interval=10,
        logfile='resource_usage.log'
    )


Starting resource monitoring for 3600s (logging every 10s to 'resource_usage.log')
→ Resource monitoring started (daemon thread).


In [11]:
from langchain_core.prompts import ChatPromptTemplate

template = """Question: {question}

Answer: Provide concise and simple answer!"""

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model

if mode == "local":
  print(chain.invoke({"question": "What is a good way to continue this sentence: 'you is a ...'? It has to by syntactically correct!"}))
elif mode == "remote":
  print(chain.invoke({"question": "What is a good way to continue this sentence: 'you is a ...'? It has to by syntactically correct!"}).content)

Answer: One way to continue the sentence could be 'you is a surprise,' though 'you are a' would be more grammatically correct.
